In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
import os
sys.path.append(os.path.join(os.path.split(os.path.abspath(''))[0], 'project/packages'))

# For writing images
if not os.path.exists("images"):
    os.mkdir("images")

In [16]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets

import data_types
import stock_data_manager
import stock_data_consumer

sdm = stock_data_manager.StockDataManager(console_logging_level='ERROR')
sdm._testing = True
sdm.run()

sdc = stock_data_consumer.StockDataConsumer(all_symbols=sdm.all_symbols,
                                            stock_categories=sdm.stock_categories,
                                            category_allocations=sdm.category_allocations,
                                            index_tracker_stocks=sdm.index_tracker_stocks,
                                            watchlist_stocks=sdm.watchlist_stocks,
                                            portfolio_stocks=sdm.portfolio_stocks,
                                            positions=sdm.positions)
sdc.run()

portfolio_stock_stats_combined = sdc.get_portfolio_stock_stats(combined=True)
portfolio_aggregated_stats = sdc.get_portfolio_aggregate_stats()
portfolio_stock_composition_stats = sdc.get_portfolio_stock_composition_stats()
portfolio_category_composition_stats = sdc.get_portfolio_category_composition_stats()
portfolio_category_composition_stats_combined = sdc.get_portfolio_category_composition_stats(combined=True)

portfolio_market_dates = sdc.portfolio_market_dates
first_date = portfolio_market_dates[0]
last_date = portfolio_market_dates[len(portfolio_market_dates)-1]
# Pick last date for any outputs
s_composition_stats = portfolio_stock_composition_stats[last_date]
c_composition_stats = portfolio_category_composition_stats[last_date]

In [17]:
min_value = portfolio_aggregated_stats['total_pct_gain'].min()
max_value = portfolio_aggregated_stats['total_pct_gain'].max()

fig = px.line(portfolio_aggregated_stats, 
              x='date', y="total_pct_gain", 
              hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                          'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                          'total_gain', 'total_pct_gain'],
              range_y=[min_value, max_value+1],
              title='Portfolio Value')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="x",
            yref="y",
            x0=first_date,
            y0=min_value,
            x1=last_date,
            y1=0,
            opacity=0.2,
            fillcolor="Red",
        )
fig.write_image("images/portfolio_value.pdf")
fig.show()

In [18]:
min_value = portfolio_aggregated_stats['total_pct_gain'].min()
max_value = portfolio_aggregated_stats['total_pct_gain'].max()

fig = px.line(portfolio_aggregated_stats, 
              x='date', y="total_pct_gain", 
              hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                          'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                          'total_gain', 'total_pct_gain'],
              color='invested_amount',
              range_y=[min_value, max_value+1],
              title='Portfolio Value by Invested Amount')
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="x",
            yref="y",
            x0=first_date,
            y0=min_value,
            x1=last_date,
            y1=0,
            opacity=0.2,
            fillcolor="Red",
        )
fig.write_image("images/portfolio_value_invested_amount.pdf")
fig.show()

In [19]:
min_value = portfolio_aggregated_stats['total_pct_gain'].min()
max_value = portfolio_aggregated_stats['total_pct_gain'].max()

fig = px.scatter(portfolio_aggregated_stats, 
                 x="date", y="total_pct_gain", 
                 trendline="lowess",
                 hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                             'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                             'total_gain', 'total_pct_gain'],
                 marginal_y='box',
                 color='invested_amount',
                 range_y=[min_value, max_value+1],
                 trendline_color_override='dodgerblue',
                 title='Portfolio Value Scatter')
fig.update_xaxes(
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="x",
            yref="y",
            x0=first_date,
            y0=min_value,
            x1=last_date,
            y1=0,
            opacity=0.2,
            fillcolor="Red",
        )
fig.write_image("images/portfolio_value_scatter.pdf")
fig.show()

In [20]:
min_value = portfolio_stock_stats_combined['total_pct_gain'].min()
max_value = portfolio_stock_stats_combined['total_pct_gain'].max()

fig = px.line(portfolio_stock_stats_combined, 
              x='date', y="total_pct_gain", 
              color="symbol",
              hover_name="symbol", 
              hover_data=['invested_amount', 'market_value', 'unrealized_gain', 
                          'unrealized_pct_gain', 'realized_gain', 'realized_pct_gain', 
                          'total_gain', 'total_pct_gain', 'open', 'close', 'high', 
                          'low', 'volume', 'quantity', 'average_cost', 'company_name'],
              range_y=[min_value, max_value+1],
              height=700,
              title='Portfolio Stocks',)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.add_shape(
        # Rectangle reference to the axes
            type="rect",
            xref="x",
            yref="y",
            x0=first_date,
            y0=min_value,
            x1=last_date,
            y1=0,
            opacity=0.2,
            fillcolor="Red",
        )
fig.write_image("images/portfolio_stocks.pdf")
fig.show()

In [21]:
fig = px.bar(s_composition_stats.sort_values(by='total_pct_gain', ascending=False),
             x="total_pct_gain", y="symbol", 
             title='Total Gain by Stock', 
             color="symbol",
             text="total_pct_gain",
             hover_data=['company_name'],
             orientation='h',
             height=700)
fig.write_image("images/total_gain_by_stock.pdf")
fig.show()

In [22]:
fig = px.bar(s_composition_stats.sort_values(by='total_pct_gain', ascending=False),
             x="composition_invested", y="symbol", 
             title='Allocation by Stock', 
             color="symbol",
             text="composition_invested",
             hover_data=['company_name'],
             orientation='h',
             height=700)
fig.write_image("images/allocation_by_stock.pdf")
fig.show()

In [23]:
fig = px.bar(c_composition_stats.sort_values(by='total_pct_gain', ascending=False),
             x="total_pct_gain", y="category", 
             title='Total Gain by Category', 
             color="category", 
             text="total_pct_gain",
             orientation='h')
fig.write_image("images/total_gain_by_category.pdf")
fig.show()

In [24]:
fig = px.bar(c_composition_stats, 
             x="desired_composition_invested_diff", y="category", 
             title='Allocation Difference', 
             color="category", 
             text="desired_composition_invested_diff", 
             hover_data=['desired_allocation'],
             orientation='h')
fig.write_image("images/allocation_difference.pdf")
fig.show()

In [25]:
fig = px.bar(c_composition_stats.sort_values(by='desired_allocation', ascending=False), 
             x="desired_allocation", 
             y="category", 
             title='Desired Allocation', 
             color="category", 
             text="desired_allocation", 
             orientation='h')
fig.write_image("images/desired_allocation.pdf")
fig.show()

In [26]:
allocation_solution = sdc.maximize_desired_allocation(date=last_date)
fig = px.bar(allocation_solution.sort_values(by='allocation_break_even_pct', ascending=False), 
             x="allocation_break_even", 
             y="category", 
             title='Allocation Break Even (%)', 
             color="category", 
             text="allocation_break_even_pct",
             hover_data=['allocation_break_even'],
             orientation='h')
fig.write_image("images/allocation_break_even.pdf")
fig.show()